# Roadway Maintenance Agreement

## Imports

In [1]:
import csv
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.utils import get_column_letter
import xlsxwriter
import re
%run C:\Users\Govs\Projects/FeatureLayerDataFrame.py

## File Paths

In [2]:
file_path = r"G:\ATD\Arterial Management\TxDOT MMA revisions - 2019"
in_file = r"\TxDOT_Roadways_Markings.csv"
out_file = r"\Markings_Asset_Total_TxDOT_update.xlsx"

## Methods

In [305]:
def count_assets(df,x,name):
    y = ["SEGMENT_ID"]
    y.extend(x)
    temp = df.filter(items=y).groupby(x).count().T
    temp['Street'] = name
    return temp

def cost_sheets(worksheet,title,total,col):
    worksheet.title = title
    worksheet["A22"].value = "Total Costs per Roadway"
    worksheet["O1"].value = total
    for i in range(2,x_end + 1):
        x = get_column_letter(i)
        worksheet["{}22".format(x)].number_format = num
        worksheet["{}22".format(x)].value = '=SUM({0}2:{0}21)'.format(x)
        for j in range(2,y_end):
            worksheet["{0}{1}".format(x,j)].value = "=VLOOKUP($A{0},'Unit Cost Table'!$A$2:$F$23,{2},FALSE) * 'Total Assets'!{1}{0}".format(j,x,col)
            worksheet["O{}".format(j)].number_format = num
            worksheet["{0}{1}".format(x,j)].number_format = num
    worksheet["O{}".format(y_end)].number_format = num

## Setup Tables

In [4]:
roadways = pd.read_csv(file_path + in_file)
table = roadways.filter(items=["Location ID","Street","Use","Segment ID List"]).query("Use == 'Y'")
table["Segment ID List"] = table["Segment ID List"].astype(str).apply(lambda x: x.rstrip().split(","))
table = table.groupby(["Location ID"]).agg({"Street": 'first',"Segment ID List": 'sum'})
table["Segment ID List"] = table["Segment ID List"].apply(tuple)
table = table.reindex(columns=['Street','Segment ID List'])

In [ ]:
results = []
for index, row in table.iterrows():
    segments = list(table['Segment ID List'])[index - 1]
    name = table.iloc[index - 1]["Street"]
    sl = FeatureLayerDataFrame('markings_short_line').query_segments('SEGMENT_ID',segments,"")
    sp = FeatureLayerDataFrame('markings_specialty_point').query_segments('SEGMENT_ID',segments,"")
    if sl is None and sp is None:
        continue
    sl = count_assets(sl,["SHORT_LINE_TYPE","SUBTYPE"],name)
    sp = count_assets(sp,["SPECIALTY_POINT_TYPE"],name)
    result = table.merge(sl,on='Street').merge(sp,on='Street').iloc[0]
    results.append(dict(result))

In [6]:
total = pd.DataFrame(results)
total.rename(columns={('CROSSWALK', 'TRANSVERSE'):'Transverse Crosswalk',('CROSSWALK', 'CONTINENTAL'):'Continental Crosswalk',
                      ('STOP_LINE', '<Null>'):'Stop Line'},inplace=True)
total.set_index(['Street'],inplace=True)
total = total.fillna(0).astype(int,errors='ignore').T
total = total[1:]

## Create Excel

In [309]:
# Convert asset totals dataframe to excel
writer = pd.ExcelWriter(file_path + out_file)
total.to_excel(writer,'Total Assets')
writer.save()
wb = load_workbook(file_path + out_file)

# Setup styling variables
white = 'ffffff'
font = 'Helvetica'
center = 'center'
num = '$#,##0.00'
width = 13
white_font = Font(name= font,size=12,color=white)
dark_font = Font(name= font,size=12,color='46494c')
red_font = Font(name= font,size=12,color='ff0000',bold=True)
center_align = Alignment(horizontal='center',vertical = 'center',wrapText=True)
right_align = Alignment(horizontal='right',vertical = 'center',wrapText=True)  
blue_fill = PatternFill("solid", fgColor="1985A1")
total_fill = PatternFill("solid", fgColor="4c5c68")
light_fill = PatternFill("solid", fgColor="DCDCDD")
gray_fill = PatternFill("solid", fgColor="c5c3c6")
white_border = Border(left = Side(border_style='thin',color=white), right = Side(border_style='thin',color=white)
                      ,top = Side(border_style='thin',color=white), bottom = Side(border_style='thin',color=white))

In [310]:
# Setup Unit Cost worksheet
ws = wb.create_sheet('Unit Cost Table')
ws.column_dimensions['A'].width = 36
with open(file_path + r'\Markings_Costs.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        ws.append(row)
wb.save(file_path + out_file)

In [311]:
# Setup Total Asset Worksheet
ws1 = wb.active
ws1.column_dimensions['A'].width = 30
ws1.row_dimensions[1].height = 110
start = 2
x_end = 14
y_end = 22

# Setup styling for total asset sheet
for col in ws1.iter_cols():
    col[0].fill = blue_fill
    col[0].font = white_font
    col[0].alignment = center_align
    ws1.column_dimensions[get_column_letter(start)].width = width
    start += 1
for cell in ws1.iter_rows():
    cell[0].alignment = right_align 
    cell[0].fill = blue_fill
    cell[0].font = white_font
    cell[0].border = white_border
    for i in range(1,x_end):
        ws1.column_dimensions[get_column_letter(i + 1)].width = width
        cell[i].alignment = center_align
        cell[i].fill = light_fill
        cell[i].font = dark_font
        if cell[i].value is 0:
            cell[i].font = red_font
        cell[i].border = white_border
ws1["O1"].value = 'Total Markings'
ws1["O1"].font = white_font
ws1["O1"].fill = total_fill
ws1["O1"].alignment = center_align
for k in range(2,y_end + 1):
    ws1["O{}".format(k)].fill = gray_fill
    ws1["O{}".format(k)].border = white_border
    ws1["O{}".format(k)].font = dark_font
    ws1["O{}".format(k)].alignment = center_align
    ws1["O{}".format(k)].value = '=SUM(B{0}:N{0})'.format(k)
    if k <= x_end:
        x = get_column_letter(k)
        ws1["{}22".format(x)].fill = gray_fill
        ws1["{}22".format(x)].border = white_border
        ws1["{}22".format(x)].font = dark_font
        ws1["{}22".format(x)].alignment = center_align
        ws1["{}22".format(x)].value = '=SUM({0}2:{0}21)'.format(x)
ws1["O22"].fill = total_fill
ws1["O22"].font = white_font
ws1["A22"].value = "Total Assets per Roadway"
ws1["A22"].fill = total_fill
ws1["A22"].border = white_border
ws1["A22"].alignment = right_align
ws1["A22"].font = white_font
wb.save(file_path + out_file)

In [312]:
# Setup Material and Labor Cost worksheets
ws2 = wb.copy_worksheet(ws1)
cost_sheets(ws2,"Total Material Costs","Total Costs per Roadway",2)
ws3 = wb.copy_worksheet(ws2)
cost_sheets(ws3,"Total Costs per Marking & Labor","Total Costs per Roadway",6)

wb.save(file_path + out_file)

In [313]:
# Reorder the sheets in the workbook
wb.active = 0
myorder = [3,2,1,0]
wb._sheets=[wb._sheets[i] for i in myorder]
wb._sheets.sort(key=lambda ws: ws.title)
wb.save(file_path + out_file)

In [314]:
wb.close()